In [1]:
import pandas as pd
import os
import re
import difflib

In [2]:
DATA_PATH = ["/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/inference", "/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/test"]

# read files

In [3]:
folder_to_file = {}
for p in DATA_PATH:
    for root, _, files in os.walk(p):
        for f in files:
            if f.endswith(".res.c") and "sanitized" not in f and "deepseek" not in f and "gpt" not in f and "claude" not in f and "spinfer" not in f:
                folder = root.split("/")[-1]
                if folder not in folder_to_file:
                    folder_to_file[folder] = []

                if f not in folder_to_file[folder]:
                    f = f.split(".res.c")[0]
                    folder_to_file[folder].append(os.path.join(root, f))

In [4]:
folder_to_file

{'EXP0-7': ['/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/inference/EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_export_btrfs_encode_fh',
  '/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/inference/EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrfs_qgroup_check_reserved_leak',
  '/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/inference/EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_disk-io___setup_root',
  '/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/inference/EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_backref_btrfs_check_shared',
  '/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/inference/EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_qgroup___btrfs_qgroup_release_data',
  '/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/inference/EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_transaction_btrfs_clean_one_deleted_snapshot',
  '/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_submission_fix/k_3/test/EXP0-7/15335331

In [5]:
folders = []
files = []
for folder, file_list in folder_to_file.items():
    for file in file_list:
        folders.append(folder)
        files.append(file)

df = pd.DataFrame({
    'folder': folders,
    'file': files
})

In [6]:
df.head(3)

,folder,file
0,EXP0-7,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...
1,EXP0-7,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...
2,EXP0-7,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   folder  230 non-null    object
 1   file    230 non-null    object
dtypes: object(2)
memory usage: 3.7+ KB


In [8]:
def open_and_read_file_content(filepath: str)->str:
    """Open the input filepath, then read the content"""
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
    except IOError as e:
        print(f"An error occurred while reading the file: {e}")
    return ""

In [9]:
df["c_before"] = df.apply(lambda x: open_and_read_file_content(x.file+".c"), axis=1)

In [10]:
df["c_after"] = df.apply(lambda x: open_and_read_file_content(x.file+".res.c"), axis=1)

# preprocessing

In [11]:
print(df.iloc[0].c_before)

static int btrfs_encode_fh(struct inode *inode, u32 *fh, int *max_len,
			   struct inode *parent)
{
	struct btrfs_fid *fid = (struct btrfs_fid *)fh;
	int len = *max_len;
	int type;
	if (parent && (len < BTRFS_FID_SIZE_CONNECTABLE)) {
		*max_len = BTRFS_FID_SIZE_CONNECTABLE;
		return FILEID_INVALID;
	} else if (len < BTRFS_FID_SIZE_NON_CONNECTABLE) {
		*max_len = BTRFS_FID_SIZE_NON_CONNECTABLE;
		return FILEID_INVALID;
	}
	len  = BTRFS_FID_SIZE_NON_CONNECTABLE;
	type = FILEID_BTRFS_WITHOUT_PARENT;
	fid->objectid = btrfs_ino(BTRFS_I(inode));
	fid->root_objectid = BTRFS_I(inode)->root->objectid;
	fid->gen = inode->i_generation;
	if (parent) {
		u64 parent_root_id;
		fid->parent_objectid = BTRFS_I(parent)->location.objectid;
		fid->parent_gen = parent->i_generation;
		parent_root_id = BTRFS_I(parent)->root->objectid;
		if (parent_root_id != fid->root_objectid) {
			fid->parent_root_objectid = parent_root_id;
			len = BTRFS_FID_SIZE_CONNECTABLE_ROOT;
			type = FILEID_BTRFS_WITH_PARENT_ROOT

In [12]:
def remove_comments(c_code:str)->str:
    # Regular expression to match both single-line and multi-line comments
    pattern = r"//.*?$|/\*.*?\*/"
    
    # Substitute the comments with an empty string
    cleaned_code = re.sub(pattern, '', c_code, flags=re.DOTALL | re.MULTILINE)
    
    return cleaned_code.strip()

In [13]:
df["c_before"] = df.c_before.apply(lambda x: remove_comments(x))
df["c_after"] = df.c_after.apply(lambda x: remove_comments(x))

In [14]:
print(df.iloc[0].c_before)

static int btrfs_encode_fh(struct inode *inode, u32 *fh, int *max_len,
			   struct inode *parent)
{
	struct btrfs_fid *fid = (struct btrfs_fid *)fh;
	int len = *max_len;
	int type;
	if (parent && (len < BTRFS_FID_SIZE_CONNECTABLE)) {
		*max_len = BTRFS_FID_SIZE_CONNECTABLE;
		return FILEID_INVALID;
	} else if (len < BTRFS_FID_SIZE_NON_CONNECTABLE) {
		*max_len = BTRFS_FID_SIZE_NON_CONNECTABLE;
		return FILEID_INVALID;
	}
	len  = BTRFS_FID_SIZE_NON_CONNECTABLE;
	type = FILEID_BTRFS_WITHOUT_PARENT;
	fid->objectid = btrfs_ino(BTRFS_I(inode));
	fid->root_objectid = BTRFS_I(inode)->root->objectid;
	fid->gen = inode->i_generation;
	if (parent) {
		u64 parent_root_id;
		fid->parent_objectid = BTRFS_I(parent)->location.objectid;
		fid->parent_gen = parent->i_generation;
		parent_root_id = BTRFS_I(parent)->root->objectid;
		if (parent_root_id != fid->root_objectid) {
			fid->parent_root_objectid = parent_root_id;
			len = BTRFS_FID_SIZE_CONNECTABLE_ROOT;
			type = FILEID_BTRFS_WITH_PARENT_ROOT

In [15]:
print(df.iloc[0].c_after)

static int btrfs_encode_fh(struct inode *inode, u32 *fh, int *max_len,
			   struct inode *parent)
{
	struct btrfs_fid *fid = (struct btrfs_fid *)fh;
	int len = *max_len;
	int type;
	if (parent && (len < BTRFS_FID_SIZE_CONNECTABLE)) {
		*max_len = BTRFS_FID_SIZE_CONNECTABLE;
		return FILEID_INVALID;
	} else if (len < BTRFS_FID_SIZE_NON_CONNECTABLE) {
		*max_len = BTRFS_FID_SIZE_NON_CONNECTABLE;
		return FILEID_INVALID;
	}
	len  = BTRFS_FID_SIZE_NON_CONNECTABLE;
	type = FILEID_BTRFS_WITHOUT_PARENT;
	fid->objectid = btrfs_ino(BTRFS_I(inode));
	fid->root_objectid = BTRFS_I(inode)->root->root_key.objectid;
	fid->gen = inode->i_generation;
	if (parent) {
		u64 parent_root_id;
		fid->parent_objectid = BTRFS_I(parent)->location.objectid;
		fid->parent_gen = parent->i_generation;
		parent_root_id = BTRFS_I(parent)->root->root_key.objectid;
		if (parent_root_id != fid->root_objectid) {
			fid->parent_root_objectid = parent_root_id;
			len = BTRFS_FID_SIZE_CONNECTABLE_ROOT;
			type = FILEID_BTRF

In [16]:
def get_loc(input_str: str)->str:
    return len(input_str.splitlines())

In [17]:
df["loc_before"] = df.c_before.apply(lambda x: get_loc(x))
df["loc_after"] = df.c_after.apply(lambda x: get_loc(x))

In [18]:
df.head(3)

,folder,file,c_before,c_after,loc_before,loc_after
0,EXP0-7,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,static int btrfs_encode_fh(struct inode *inode...,static int btrfs_encode_fh(struct inode *inode...,35,35
1,EXP0-7,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,void btrfs_qgroup_check_reserved_leak(struct i...,void btrfs_qgroup_check_reserved_leak(struct i...,23,23
2,EXP0-7,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,static void __setup_root(struct btrfs_root *ro...,static void __setup_root(struct btrfs_root *ro...,63,63


In [19]:
def get_diff(file_a: str, file_b: str, n_context: int)->str:
    """Get the diff between file_a and file_b using difflib with n_context length"""
    lines_a = file_a.splitlines()
    lines_b = file_b.splitlines()        
    # Generate the unified diff
    diff = difflib.unified_diff(
        lines_a,
        lines_b,
        fromfile="initial",
        tofile="final",
        lineterm='',
        n=n_context
    )

    final_diff = []
    for l in diff:
        if l.startswith("---") or l.startswith("+++"):
            continue
        elif l.startswith("@@ ") and l.endswith(" @@"):
            final_diff.append("<unmodified lines>")
        else:
            final_diff.append(l)

    if final_diff[0] == "<unmodified lines>":
        final_diff = final_diff[1:]

    # Convert the diff generator to a single string
    diff_text = '\n'.join(final_diff)
    return diff_text

In [20]:
df["diff_full"] = df.apply(lambda x: get_diff(x.c_before, x.c_after, 10000000), axis=1)
# df["diff_only"] = df.apply(lambda x: get_diff(x.c_before, x.c_after, 0), axis=1)

In [21]:
df["loc_diff_full"] = df.diff_full.apply(lambda x: get_loc(x))
# df["loc_diff_only"] = df.diff_only.apply(lambda x: get_loc(x))

In [22]:
df.sample(n=50)

,folder,file,c_before,c_after,loc_before,loc_after,diff_full,loc_diff_full
92,snd_soc,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,static int mt2701_afe_i2s_path_shutdown(struct...,static int mt2701_afe_i2s_path_shutdown(struct...,33,31,static int mt2701_afe_i2s_path_shutdown(struc...,34
208,dasd_smalloc,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,static struct dasd_ccw_req *dasd_diag_build_cp...,static struct dasd_ccw_req *dasd_diag_build_cp...,70,71,static struct dasd_ccw_req *dasd_diag_build_c...,72
63,sock_poll_wait-84,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,"static __poll_t caif_poll(struct file *file, s...","static __poll_t caif_poll(struct file *file, s...",16,16,"static __poll_t caif_poll(struct file *file, ...",17
155,kees_timer1,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,int fas216_init(struct Scsi_Host *host)\n{\n\t...,int fas216_init(struct Scsi_Host *host)\n{\n\t...,29,27,int fas216_init(struct Scsi_Host *host)\n {\n...,30
117,uartlite,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,static int ulite_get_poll_char(struct uart_por...,static int ulite_get_poll_char(struct uart_por...,7,7,static int ulite_get_poll_char(struct uart_po...,9
30,tcf_block_get-61,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,"static int atm_tc_init(struct Qdisc *sch, stru...","static int atm_tc_init(struct Qdisc *sch, stru...",24,24,"static int atm_tc_init(struct Qdisc *sch, str...",25
209,dasd_smalloc,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,static struct dasd_ccw_req *dasd_fba_build_cp_...,static struct dasd_ccw_req *dasd_fba_build_cp_...,88,89,static struct dasd_ccw_req *dasd_fba_build_cp...,90
111,ttm_bo_init-60,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,"int hibmc_bo_create(struct drm_device *dev, in...","int hibmc_bo_create(struct drm_device *dev, in...",28,28,"int hibmc_bo_create(struct drm_device *dev, i...",29
141,early_memunmap,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,static void __init memblock_x86_reserve_range_...,static void __init memblock_x86_reserve_range_...,11,11,static void __init memblock_x86_reserve_range...,12
71,random_ether_addr-84,/media/hdd2/yusuf/SMPL-How-Far-Are-We/emse_sub...,static void lan78xx_init_mac_address(struct la...,static void lan78xx_init_mac_address(struct la...,42,42,static void lan78xx_init_mac_address(struct l...,43


In [23]:
import code_tokenize as ctok
    
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    if string:
        num_tokens = len(ctok.tokenize(string, lang="c", syntax_error="ignore"))
        return num_tokens
    return 0

df["num_tokens"] = df["diff_full"].apply(lambda x: num_tokens_from_string(x))

In [24]:
df.num_tokens

0      239
1      175
2      536
3      324
4      188
      ... 
225    395
226    395
227    854
228    466
229    764
Name: num_tokens, Length: 230, dtype: int64

In [25]:
df.drop(columns=["c_before", "c_after"], inplace=True)
# df.drop(columns=["diff_full"], inplace=True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   folder         230 non-null    object
 1   file           230 non-null    object
 2   loc_before     230 non-null    int64 
 3   loc_after      230 non-null    int64 
 4   diff_full      230 non-null    object
 5   loc_diff_full  230 non-null    int64 
 6   num_tokens     230 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 12.7+ KB


In [31]:
df.folder.unique()

array(['EXP0-7', 'tcf_block_get-61', 'sock_poll_wait-84',
       'random_ether_addr-84', 'snd_soc', 'ttm_bo_init-60', 'uartlite',
       'perf_evlist__mmap-69', 'early_memunmap', 'kees_timer1',
       'dma_pool_alloc-52', 'free_bootmem-77', 'tcaction', 'dasd_smalloc'],
      dtype=object)

In [33]:
import pandas as pd

# Assuming your dataframe is already loaded as 'df'
# If not, you would load it here

# Calculate the statistics
# First, get the folder counts
folder_counts = df['folder'].value_counts()

# Create a result dataframe with folder as the index
result = pd.DataFrame(index=folder_counts.index)

# Number of functions (files) per folder
result['num_of_function'] = df.groupby('folder')['file'].nunique()

# Number of variants using the specified values
variant_counts = {
    'kees_timer1': 3,
    'EXP0-7': 2,
    'snd_soc': 2,
    'tcf_block_get-61': 2,
    'dasd_smalloc': 2,
    'sock_poll_wait-84': 1,
    'ttm_bo_init-60': 1,
    'random_ether_addr-84': 1,
    'perf_evlist__mmap-69': 1,
    'uartlite': 1,
    'early_memunmap': 1,
    'dma_pool_alloc-52': 1,
    'free_bootmem-77': 1,
    'tcaction': 1
}

# Fill the variants column, default to 0 for any folder not in the variant_counts dictionary
result['num_of_variant'] = pd.Series(variant_counts).reindex(index=result.index).fillna(0).astype(int)

# Average LoC diff per folder
result['avg_LoC_diff'] = df.groupby('folder')['loc_diff_full'].mean()

# Average tokens per folder
result['avg_tokens'] = df.groupby('folder')['num_tokens'].mean()

# Max tokens per folder
result['max_tokens'] = df.groupby('folder')['num_tokens'].max()

# Reset index to make folder a column
result = result.reset_index().rename(columns={'index': 'folder'})

# Display the results
print(result)

# If you want to round the decimal places
result_rounded = result.copy()
result_rounded['avg_LoC_diff'] = result_rounded['avg_LoC_diff'].round(2)
result_rounded['avg_tokens'] = result_rounded['avg_tokens'].round(2)
print("\nResults with rounded averages:")
print(result_rounded)

                  folder  num_of_function  num_of_variant  avg_LoC_diff  \
0       tcf_block_get-61               33               2     57.666667   
1                snd_soc               26               2     28.923077   
2            kees_timer1               25               3     62.920000   
3           dasd_smalloc               24               2     83.958333   
4                 EXP0-7               21               2     71.190476   
5   random_ether_addr-84               15               1     66.066667   
6   perf_evlist__mmap-69               14               1     93.357143   
7      sock_poll_wait-84               13               1     24.230769   
8        free_bootmem-77               13               1     21.076923   
9               tcaction               11               1     57.272727   
10        early_memunmap               11               1     23.454545   
11        ttm_bo_init-60                9               1     32.111111   
12     dma_pool_alloc-52 

In [35]:
result_rounded.to_csv("dataset_stats.csv", index=False)